In [1]:
import numpy as np
import hddCRP.markovModeling as mm

In [34]:
# make sequence to fit (totally random)
K = 3
rng = np.random.Generator(np.random.MT19937(1023))
seq = rng.integers(0,K,size=(100))
symbols = range(K)


In [45]:
depth = 0 # how far back in time to care about dependencies (1=only look at previous action)

C = mm.get_transition_count(seq, depth=depth, symbols=symbols)
P = mm.get_transition_probabilities(C) # the plugin estimator for the transitions

# I also have some Bayesian estimation options
P1 = mm.get_transition_probabilities(C, alpha=1, rng=None) # add a regularization term in a simple categorical-dirichlet model for posterior mean estimates
P2 = mm.get_transition_probabilities(C, alpha=1, rng=rng) # can also draw P from the posterior over P under the categorical-dirichlet model
                                                          # This can help us figure out if the uncertainty in our estimates matters
P3 = mm.get_transition_probabilities_hierarchical(C, alpha=1, rng=None) #changes up the regularization using a very quick hierarchical scheme
P4 = mm.get_transition_probabilities_hierarchical(C, alpha=1, rng=rng) # 


array([0.31131383, 0.2977318 , 0.39095437])

In [46]:
initial_state = [0,1] # I ended up requiring an initial state to start the simulation - otherwise I wouldn't be sure how you'd want to begin
simulated_seqs = mm.simulate_from_model(P, initial_state, sim_length=100, rng=rng, nsims=10)
simulated_seqs # each column is one simulation (first steps are set to initial_state)

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [2, 2, 0, 2, 1, 1, 1, 0, 1, 2],
       [1, 2, 1, 1, 0, 0, 1, 1, 0, 2],
       [2, 0, 1, 0, 2, 2, 1, 2, 0, 2],
       [2, 2, 0, 0, 2, 0, 0, 1, 2, 2],
       [2, 0, 2, 0, 2, 2, 0, 2, 2, 1],
       [0, 1, 1, 0, 2, 0, 2, 1, 1, 2],
       [2, 1, 0, 0, 2, 2, 0, 1, 0, 2],
       [2, 2, 0, 0, 2, 0, 2, 1, 2, 2],
       [2, 2, 2, 0, 1, 0, 1, 1, 2, 1],
       [0, 0, 0, 1, 1, 0, 1, 1, 0, 1],
       [1, 0, 0, 1, 1, 1, 2, 2, 1, 2],
       [0, 1, 2, 2, 1, 0, 0, 0, 0, 1],
       [0, 2, 0, 2, 1, 0, 0, 2, 1, 2],
       [0, 0, 0, 2, 0, 0, 1, 2, 1, 0],
       [0, 0, 1, 2, 0, 0, 2, 1, 1, 2],
       [1, 1, 0, 1, 0, 0, 1, 2, 2, 2],
       [0, 1, 0, 1, 0, 1, 0, 2, 0, 0],
       [0, 1, 2, 0, 0, 2, 1, 1, 1, 2],
       [1, 2, 0, 0, 0, 0, 1, 1, 0, 2],
       [1, 0, 0, 2, 2, 2, 0, 1, 2, 1],
       [1, 2, 2, 1, 0, 1, 1, 2, 2, 2],
       [2, 1, 2, 1, 2, 1, 2, 2, 0, 2],
       [2, 1, 2, 2, 1, 1, 0, 1, 2, 0],
       [2, 0, 2, 0, 2, 0,

In [47]:
# with multiple sessions, there's a boundary problem at the start of each session.
# Conditional counts can still be extracted by summation over each session
seqs = [];
for ii in range(2):
    seqs += [rng.integers(0,K,size=(100))]

C = 0
for ss in seqs:
    C = C + mm.get_transition_count(ss, depth=depth, symbols=symbols)
P = mm.get_transition_probabilities(C) # the plugin estimator for the transitions

[array([1, 2, 0, 1, 0, 0, 1, 1, 1, 2, 0, 0, 2, 1, 1, 1, 0, 1, 1, 2, 2, 2,
        0, 1, 1, 2, 2, 1, 1, 0, 2, 1, 0, 0, 0, 2, 1, 2, 0, 0, 1, 1, 1, 1,
        2, 1, 1, 1, 2, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 2, 2, 0, 2, 2, 0, 0,
        0, 1, 0, 0, 1, 2, 1, 2, 0, 2, 0, 1, 0, 0, 1, 0, 2, 2, 1, 2, 1, 2,
        1, 0, 1, 0, 1, 0, 2, 1, 0, 2, 1, 2]),
 array([2, 0, 0, 1, 1, 1, 1, 0, 1, 1, 2, 2, 1, 0, 0, 1, 1, 1, 2, 1, 1, 0,
        2, 2, 2, 2, 2, 0, 1, 0, 0, 0, 2, 0, 1, 2, 1, 2, 0, 0, 2, 1, 0, 2,
        0, 2, 0, 1, 2, 2, 2, 0, 2, 2, 1, 2, 1, 1, 2, 0, 2, 0, 1, 2, 2, 2,
        1, 0, 2, 2, 1, 2, 1, 1, 0, 2, 1, 1, 2, 0, 0, 1, 2, 0, 2, 1, 2, 2,
        1, 1, 0, 1, 1, 1, 2, 2, 2, 2, 2, 0])]